In [11]:
import json
import gzip
import numpy as np
import string
import random
import operator
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
# from pylmnn.lmnn import LargeMarginNearestNeighbor as LMNN
# from pylmnn.plots import plot_comparison
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm

def parseData(file):
    for l in open(file,'r'):
        yield json.loads(l)
        
def remove_punctuation(text):
    return ''.join([c.lower() for c in text if c not in set(string.punctuation)])

np.random.seed(5)

In [12]:
data = list(parseData('./renttherunway_final_data.json'))

In [13]:
print(data[0])

{'fit': 'fit', 'user_id': '420272', 'bust size': '34d', 'item_id': '2260466', 'weight': '137lbs', 'rating': '10', 'rented for': 'vacation', 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.", 'body type': 'hourglass', 'review_summary': 'So many compliments!', 'category': 'romper', 'height': '5\' 8"', 'size': 14, 'age': '28', 'review_date': 'April 20, 2016'}


In [14]:
dataNeed = []
fit = set()
for d in data:
    newd = dict()
    newd['user_id'] = d['user_id']
    newd['item_id'] = d['item_id']
    newd['size'] = d['size']
    newd['fit'] = d['fit']
    if 'rating' not in d or d['rating'] == None:
        continue
    newd['rating'] = (int)(d['rating']) / 2
    if d['fit'] == None:
        continue
    fit.add(d['fit'])
    dataNeed.append(newd)

In [15]:
print(dataNeed[0])

{'user_id': '420272', 'item_id': '2260466', 'size': 14, 'fit': 'fit', 'rating': 5.0}


In [16]:
random.seed(1)
random.shuffle(data)

train_data = dataNeed[:int(0.8*len(dataNeed))]
val_data = dataNeed[int(0.8*len(dataNeed)):int(0.9*len(dataNeed))]
test_data = dataNeed[int(0.9*len(dataNeed)):]

len(train_data), len(val_data), len(test_data)

(153969, 19246, 19247)

In [21]:
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
fitnessPerUser = defaultdict(list)
fitnessPerItem = defaultdict(list)

for r in train_data:
    ratingsPerItem[r['item_id'] + '|' + str(r['size'])].append((r['user_id'], r['rating']))
    ratingsPerUser[r['user_id']].append((r['item_id'] + '|' + str(r['size']), r['rating']))
    if r['fit'] == 'fit':
        fitnessPerUser[r['user_id']].append((r['item_id'] + '|' + str(r['size']), 1))
        fitnessPerItem[r['item_id'] + '|' + str(r['size'])].append((r['user_id'], 1))
    if not r['fit'] == 'fit':
        fitnessPerUser[r['user_id']].append((r['item_id'] + '|' + str(r['size']), -1))
        fitnessPerItem[r['item_id'] + '|' + str(r['size'])].append((r['user_id'], -1))

In [1]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [2]:
def Cosine(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    if (len(s1) == 0 or len(s2) == 0):
        return 0
    nom = 0
    s1_book = [i[0] for i in s1]
    s2_book = [i[0] for i in s2]
    s1_rate = [i[1] for i in s1]
    s2_rate = [i[1] for i in s2]
    for b,r in s1:
        if b in s2_book:
            nom += s1_rate[s1_book == b] * s2_rate[s2_book == b]
    den = np.sqrt(np.sum(np.square(s1_rate))) * np.sqrt(np.sum(np.square(s2_rate)))
    if den != 0:
        return nom / den
    return 0

In [3]:
def Perlson(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    if (len(s1) == 0 or len(s2) == 0):
        return 0
    nom = 0
    s1_book = [i[0] for i in s1]
    s2_book = [i[0] for i in s2]
    s1_rate = [i[1] for i in s1]
    s2_rate = [i[1] for i in s2]
    s1_avg = np.sum(s1_rate) / len(s1)
    s2_avg = np.sum(s2_rate) / len(s2)
    s1_rate = [i - s1_avg for i in s1_rate]
    s1_rate = [i - s2_avg for i in s2_rate]
    for b,r in s1:
        if b in s2_book:
            nom += s1_rate[s1_book == b] * s2_rate[s2_book == b]
    den = np.sqrt(np.sum(np.square(s1_rate))) * np.sqrt(np.sum(np.square(s2_rate)))
    if den != 0:
        return nom / den
    return 0

In [4]:
def maxSimUser(u, b):
    maxSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Jaccard(users,set(ratingsPerItem[b2]))
        if sim > maxSim:
            maxSim = sim
    return maxSim

In [5]:
def maxSimItem(b, u):
    maxSim = 0
    books = set(ratingsPerUser[u])
    for u2,_ in ratingsPerItem[b]:
        sim = Jaccard(books,set(ratingsPerUser[u2]))
        if sim > maxSim:
            maxSim = sim
    return maxSim

In [6]:
def maxCosSimUser(u, b):
    maxCosSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Cosine(users,set(ratingsPerItem[b2]))
        if sim > maxCosSim:
            maxCosSim = sim
    return maxCosSim

In [7]:
def maxCosSimItem(b, u):
    maxCosSim = 0
    books = set(ratingsPerUser[u])
    for u2,_ in ratingsPerItem[b]:
        sim = Cosine(books,set(ratingsPerUser[u2]))
        if sim > maxCosSim:
            maxCosSim = sim
    return maxCosSim

In [8]:
def maxPerlSimUser(u, b):
    maxPerlSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Perlson(users,set(ratingsPerItem[b2]))
        if sim > maxPerlSim:
            maxPerlSim = sim
    return maxPerlSim

In [9]:
def maxPerlSimItem(b, u):
    maxPerlSim = 0
    books = set(ratingsPerUser[u])
    for u2,_ in ratingsPerItem[b]:
        sim = Perlson(books,set(ratingsPerUser[u2]))
        if sim > maxPerlSim:
            maxPerlSim = sim
    return maxPerlSim

In [26]:
validData = []
for r in val_data:
    u, i = r['user_id'], r['item_id']
    maxUser = maxSimUser(u, i)
    maxItem = maxSimBook(i, u)
    feature = [1, maxUser, maxItem]
    if r['fit'] == 'fit':
        validData.append([feature, 1])
    else:
        validData.append([feature, 0])

In [27]:
random.shuffle(validData)
validX = []
validY = []

for data in validData:
    validX.append(data[0])
    validY.append(data[1])

In [29]:
clf = LogisticRegression(C=1,solver='lbfgs').fit(validX, validY)

In [30]:
validscore = clf.score(validX, validY)
print(validscore)

0.7391665800685857
